In [1]:
library(swat)
library(ggplot2)
library(reshape2)
library(dplyr)

SWAT 1.6.1
Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang

Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [2]:
Sys.setenv(CAS_CLIENT_SSL_CA_LIST = "/opt/sas/viya/config/etc/SASSecurityCertificateFramework/cacerts/trustedcerts.pem")
conn <- CAS('frasepviya35smp.cloud.com', 5570)

NOTE: Connecting to CAS and generating CAS action functions for loaded
      action sets...
NOTE: To generate the functions with signatures (for tab completion), set 
      options(cas.gen.function.sig=TRUE).


In [3]:
# Activate metric tracing and other session parameters
cas.sessionProp.setSessOpt(conn, metrics=TRUE, timeout=1800, caslib='casuser')

NOTE: 'CASUSER(viyademo01)' is now the active caslib.
NOTE: Action 'sessionProp.setSessOpt' used (Total process time):
NOTE:       real time               0.003192 seconds
NOTE:       cpu time                0.003140 seconds (98.37%)
NOTE:       total nodes             1 (32 cores)
NOTE:       total memory            125.75G
NOTE:       memory                  394.03K (0.00%)


list()

In [4]:
Sys.time()
tbl <- defCasTable(conn, tablename="sample_350columns", caslib = "PUBLIC")
tblR <- to.casDataFrame(tbl)
Sys.time()

[1] "2021-05-10 18:17:23 UTC"

NOTE: Executing action 'table.columnInfo'.
NOTE: Action 'table.columnInfo' used (Total process time):
NOTE:       real time               0.004999 seconds
NOTE:       cpu time                0.004826 seconds (96.54%)
NOTE:       total nodes             1 (32 cores)
NOTE:       total memory            125.75G
NOTE:       memory                  1.53M (0.00%)
NOTE: Executing action 'table.fetch'.
NOTE: Action 'table.fetch' used (Total process time):
NOTE:       real time               2.468240 seconds
NOTE:       cpu time                2.468269 seconds (100.00%)
NOTE:       total nodes             1 (32 cores)
NOTE:       total memory            125.75G
NOTE:       memory                  23.63M (0.02%)


[1] "2021-05-10 18:17:33 UTC"

In [9]:
dim(tblR)

[1] 32556   350

In [10]:
cas.session.endSession(conn)

NOTE: Executing action 'session.endSession'.
NOTE: Action 'session.endSession' used (Total process time):
NOTE:       real time               0.000357 seconds
NOTE:       cpu time                0.000328 seconds (91.88%)
NOTE:       total nodes             1 (32 cores)
NOTE:       total memory            125.75G
NOTE:       memory                  208.81K (0.00%)


list()